# Import

In [1]:
import json
from jovis_model.config import Config
from run import ModelRunner

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Chatbot

In [2]:
params = {
    "pkg": "llm",
    "task": "chat",
    "use_hf_model": True,
    "params": {
        "hf_name": "meta-llama/Meta-Llama-3-8B-Instruct"
    }
}

In [3]:
config = Config(**params)
runner = ModelRunner(
    config=config,
    mode="inference"
)

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.22it/s]


In [7]:
dialogs = [
    {"role": "user", "content": "What is you name?"}
]

In [8]:
out = runner.run(
    sample_inputs=dialogs
)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [10]:
print(out)

I don't have a personal name. I am an AI designed to assist and communicate with users, and I don't have a personal identity or consciousness. I exist solely to provide information, answer questions, and help with tasks to the best of my abilities. I am often referred to as a "chatbot" or a "virtual assistant," but I don't have a personal name like a human would.
